## Wikipedia Article Retrieval Tool

This section defines a simple Python function, `get_article`, that uses the `wikipedia` Python package to search for and retrieve the content of a Wikipedia article based on a search term. This function will be used as a tool for the agent to fetch up-to-date information from Wikipedia.

In [1]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

## Example: Fetching and Previewing Wikipedia Articles

Here, we demonstrate how to use the `get_article` function to retrieve and preview the content of Wikipedia articles for various search terms, such as "Avengers: Doomsday", "Nezha 2", "History of Malaysia", and "Iron Man". Only a preview of the article content is printed for brevity.

In [ ]:
article = get_article("Avengers: Doomsday")
print(article[:1000]) # article is very long, so let's just print a preview

In [ ]:
article = get_article("Nezha 2")
print(article[:500]) # article is very long, so let's just print a preview

In [ ]:
article = get_article("History of Malaysia")
print(article[:3000]) #article is super long so let's just print a preview

In [ ]:
article = get_article("Iron Man")
print(article[:1000]) #article is super long so let's just print a preview

## Tool Schema Definition for Agent Use

This cell defines a tool schema dictionary, `article_search_tool`, which describes the Wikipedia retrieval tool in a format compatible with agentic LLM APIs (like Anthropic Claude). The schema includes the tool's name, description, input schema, and required fields.

In [2]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

## Setting Up Anthropic Client and Making a Tool-Use Request

This section initializes the Anthropic API client and demonstrates how to send a message to the Claude model, asking a question that may require tool use (e.g., "What is the box office for Nezha 2?"). The tool schema is provided to the model, enabling it to call the Wikipedia tool if needed.

In [18]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [{"role": "user", "content": "What is the box office for Nezha 2"}]

response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

In [19]:
response.content

[TextBlock(text='I\'d be happy to look up the box office information for "Nezha 2" for you. Let me search for that information on Wikipedia.', type='text'),
 ToolUseBlock(id='toolu_01Nn5gEP6BzD27vpg3CcxXQL', input={'search_term': 'Nezha 2'}, name='get_article', type='tool_use')]

## Appending the Assistant's Tool Use to the Conversation

This cell appends the assistant's tool use response to the ongoing conversation history, preparing for the next step in the agentic interaction.

In [6]:
messages.append({"role": "assistant", "content": response.content})

## Extracting Tool Use Information

This section extracts the tool name and input parameters from the model's tool use response, so that the tool can be called programmatically.

In [7]:
# This is a simple, but brittle way of getting the tool use information
# We're simply taking the last block from Claude's response.
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("Tool name: ", tool_name)
print("Tool input", tool_input)

Tool name:  get_article
Tool input {'search_term': 'Nezha 2'}


## Executing the Tool and Returning Results

Here, the code checks if the model requested the Wikipedia tool, executes the tool with the provided input, and prints a preview of the Wikipedia article content.

In [8]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"Searching Wikipedia for {search_term}")
    print("WIKIPEDIA PAGE CONTENT:")
    print(wiki_result[:500]) #just printing a small bit of the article because it's so long

Searching Wikipedia for Nezha 2
WIKIPEDIA PAGE CONTENT:
Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin's 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, although he ca


## Constructing a Tool Result Message

This cell shows how to construct a message containing the tool result, formatted according to the agentic API's expectations, so it can be sent back to the model.

In [9]:
{
  "role": "user",
  "content": [
    {
      "type": "tool_result",
      "tool_use_id": "toolu_01A09q90qw90lq917835lq9",
      "content": "The result of actually calling the tool goes here"
    }
  ]
}

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_01A09q90qw90lq917835lq9',
   'content': 'The result of actually calling the tool goes here'}]}

## Sending the Tool Result to the Conversation

The tool result message is appended to the conversation history, allowing the agent to use the tool's output in its next response.

In [10]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [11]:
tool_response

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_01DnHK4Tdbgh5khz7em7xmt1',
   'content': 'Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin\'s 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, although he carries Ao Bing’s spirit within. Ne Zha calls on this spirit in his fight against wicked Master Shen.\nNe Zha 2 was released in theaters across China on 29 January 2025, coinciding with the first day of the Chinese New Year. Like its predecessor, the film received highly positive reviews from critics for its animation, action sequences, worldbuilding, and emotional core, and ac

In [12]:
messages.append(tool_response)

In [13]:
follow_up_response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

In [14]:
follow_up_response.content[0].text

"Based on the information from the Wikipedia article, I can provide you with details about the box office performance of Nezha 2 (also known as Ne Zha 2):\n\n# Box Office for Nezha 2:\n\nAs of the article's publication, Nezha 2 has grossed more than $2.2 billion globally against a production budget of US$80 million. The film broke numerous box office records, including:\n\n- It became the highest-grossing film in a single box office territory\n- Globally became the highest-grossing animated film\n- Became the highest-grossing non-English language film (being the first to gross over $2 billion)\n- The first animated film in history to cross the $2 billion mark\n- Currently ranks as the highest-grossing film of 2025\n- The fifth-highest-grossing film of all time\n- The highest-selling animated film in terms of ticket sales\n\nThe film experienced incredible momentum in China:\n- Reached 1.1 billion yuan in just three days\n- Exceeded 2 billion yuan on the fourth day\n- Exceeded 3 billion

## Model Follow-up: Final Answer Generation

This section sends the updated conversation (including the tool result) back to the model, prompting it to generate a final answer that incorporates the information retrieved from Wikipedia.

In [21]:

def answer_question(question):
    messages = [{"role": "user", "content": question}]

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    if(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")
            wiki_result = get_article(search_term) #get wikipedia article content
            #construct our tool_result message
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": wiki_result
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        print("Claude did not call our tool")
        print(response.content[0].text)

## Agentic Tool Use: Full Question-Answering Loop

This cell defines a reusable function, `answer_question`, that demonstrates the full agentic tool-use loop: sending a question to the model, detecting tool use, executing the tool, sending the result, and printing the model's final answer.

In [22]:
answer_question("What are the names of all the Avengers films in the Marvel Cinematic Universe?")

Claude wants to get an article for Avengers (Marvel Cinematic Universe)
Claude's final answer:
Based on the Wikipedia article, here's a comprehensive list of all the Avengers films in the Marvel Cinematic Universe:

1. **The Avengers** (2012) - The first film that brought the original six Avengers together (Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye)

2. **Avengers: Age of Ultron** (2015) - The second Avengers film featuring the team battling the artificial intelligence Ultron

3. **Avengers: Infinity War** (2018) - The third Avengers film where the team faces Thanos who seeks to collect all six Infinity Stones

4. **Avengers: Endgame** (2019) - The fourth Avengers film that concludes the Infinity Saga and depicts the team's attempt to reverse Thanos' actions

Additionally, two future Avengers films have been announced:

5. **Avengers: Doomsday** (planned for 2026)

6. **Avengers: Secret Wars** (planned for 2027)

It's worth noting that Captain America: Civil War (